#  Part 0: Project Setup & Data Generation
*Before we start analyzing, let's generate a realistic "E-Commerce" dataset containing sales, dates, and customer info. This ensures the notebook runs reproducibly without needing external files.*

In [12]:
# Python Setup: Generating the Dataset
import pandas as pd
import numpy as np
import os

# 1. Define the dataset parameters
np.random.seed(42) # Ensures we get the same random numbers every time
row_count = 100
dates = pd.date_range(start='2024-01-01', periods=row_count)
categories = ['Electronics', 'Clothing', 'Home', 'Toys']
cities = ['Berlin', 'Munich', 'Hamburg', 'Cologne']

# 2. Create the DataFrame
data = {
    'Date': np.random.choice(dates, row_count),
    'Order_ID': range(1001, 1001 + row_count),
    'Category': np.random.choice(categories, row_count),
    'Sales': np.random.randint(50, 500, row_count),
    'Quantity': np.random.randint(1, 10, row_count),
    'City': np.random.choice(cities, row_count)
}

df = pd.DataFrame(data)

# 3. Introduce "Messy" Data (Real World Simulation)
# Adding NULL values to 'Category' to practice cleaning later
df.loc[0:4, 'Category'] = np.nan 

# 4. Save to CSV
csv_name = 'ecommerce_data.csv'
df.to_csv(csv_name, index=False)

print(f"✅ Success! '{csv_name}' has been generated with {row_count} rows.")
print("   We are ready to start the comparison below!")

✅ Success! 'ecommerce_data.csv' has been generated with 100 rows.
   We are ready to start the comparison below!


# Part 1: Data Exploration & Filtering
*Goal: Getting to know your dataset—peeking, slicing, and dicing.*

| Task | 🟢 Excel | 🔵 SQL | 🟡 Python (Pandas) |
| :--- | :--- | :--- | :--- |
| **View Data (First Look)** | Open file | `SELECT * FROM table LIMIT 5;` | `df.head(5)` |
| **Select Columns** | Click Header | `SELECT City, Sales FROM table;` | `df[['City', 'Sales']]` |
| **Basic Filtering** | Filter > 'Berlin' | `WHERE City = 'Berlin'` | `df[df['City'] == 'Berlin']` |
| **Multiple Filters (AND)** | Filter A & Filter B | `WHERE City = 'Berlin' AND Sales > 100` | `df[(df['City'] == 'Berlin') & (df['Sales'] > 100)]` |
| **Sorting** | Sort Z to A | `ORDER BY Sales DESC` | `df.sort_values(by='Sales', ascending=False)` |
| **Unique Values** | Remove Duplicates | `SELECT DISTINCT City FROM table;` | `df['City'].unique()` |
| **Count Rows** | Count Status Bar | `SELECT COUNT(*) FROM table;` | `len(df)` |

In [13]:
# Python Real Example: Exploring our E-Commerce Data
import pandas as pd

# 1. Load Data
df = pd.read_csv('ecommerce_data.csv')

# 2. Peek at the data (First 5 rows)
print("--- 1. First 5 Rows of Data ---")
print(df.head())

# 3. Filter: Find all orders from 'Berlin' with Sales over 200
berlin_high_sales = df[ (df['City'] == 'Berlin') & (df['Sales'] > 200) ]
print(f"\n--- 2. High Value Orders in Berlin: {len(berlin_high_sales)} orders found ---")
print(berlin_high_sales.head(3))

# 4. Sort: Who bought the most items (Quantity)?
sorted_data = df.sort_values(by='Quantity', ascending=False)
print("\n--- 3. Top 3 Highest Quantity Orders ---")
print(sorted_data[['Date', 'City', 'Quantity', 'Sales']].head(3))

--- 1. First 5 Rows of Data ---
         Date  Order_ID Category  Sales  Quantity     City
0  2024-02-21      1001      NaN     94         4   Berlin
1  2024-04-02      1002      NaN    111         3  Cologne
2  2024-01-15      1003      NaN    490         3  Hamburg
3  2024-03-12      1004      NaN    183         3  Cologne
4  2024-03-01      1005      NaN    333         4   Munich

--- 2. High Value Orders in Berlin: 12 orders found ---
          Date  Order_ID Category  Sales  Quantity    City
25  2024-03-16      1026     Home    408         5  Berlin
27  2024-01-22      1028     Home    460         7  Berlin
33  2024-04-01      1034     Toys    283         4  Berlin

--- 3. Top 3 Highest Quantity Orders ---
          Date     City  Quantity  Sales
55  2024-01-09  Cologne         9    236
14  2024-01-22   Munich         9    280
49  2024-02-08   Berlin         9    436


In [14]:
-- SQL Equivalent
If we were running this on a database, here is the syntax for the operations above:

-- 1. View Data (First 5 rows)
SELECT * FROM orders 
LIMIT 5;

-- 2. Filter (Berlin & Sales > 200)
SELECT * FROM orders 
WHERE City = 'Berlin' AND Sales > 200;

-- 3. Sort by Quantity (Highest first)
SELECT Date, City, Quantity, Sales 
FROM orders 
ORDER BY Quantity DESC 
LIMIT 3;

SyntaxError: invalid syntax (130716354.py, line 1)

# Part 2: Data Aggregation & Grouping
*Goal: Moving from detailed rows to summary statistics (The "Pivot Table" logic).*

| Task | 🟢 Excel | 🔵 SQL | 🟡 Python (Pandas) |
| :--- | :--- | :--- | :--- |
| **Sum per Group** | Pivot Table (Rows: City, Values: Sum of Sales) | `SELECT City, SUM(Sales) FROM table GROUP BY City;` | `df.groupby('City')['Sales'].sum()` |
| **Count per Group** | Pivot Table (Rows: Category, Values: Count of ID) | `SELECT Category, COUNT(*) FROM table GROUP BY Category;` | `df.groupby('Category')['Order_ID'].count()` |
| **Average** | Pivot Table (Values: Average of Sales) | `SELECT AVG(Sales) FROM table;` | `df['Sales'].mean()` |
| **Multiple Aggregations** | Pivot Table (Values: Sum & Avg) | `SELECT SUM(Sales), AVG(Price) ...` | `df.groupby('City').agg({'Sales': 'sum', 'Price': 'mean'})` |

In [ ]:
# Python Real Example: Aggregating Data
import pandas as pd
from IPython.display import display

# 1. Load Data (Bu satırı ekledik ki hata vermesin)
df = pd.read_csv('ecommerce_data.csv')

# 2. Total Sales per City (Which city makes the most money?)
# We use .to_frame() to make it look prettier in the output
city_sales = df.groupby('City')['Sales'].sum().sort_values(ascending=False)
print("--- 1. Total Revenue by City ---")
display(city_sales.to_frame())

# 3. Category Performance (Avg Price & Total Quantity Sold)
category_stats = df.groupby('Category').agg({
    'Sales': 'mean',      # Average Price
    'Quantity': 'sum'     # Total Volume
})
print("\n--- 2. Category Performance ---")
display(category_stats)

# 4. Simple Stat: Overall Average Order Value
avg_order = df['Sales'].mean()
print(f"\n--- 3. Overall Average Order Value: ${avg_order:.2f} ---")

In [ ]:
-- SQL Equivalent

-- 1. Total Revenue by City
SELECT City, SUM(Sales) as Total_Revenue
FROM orders
GROUP BY City
ORDER BY Total_Revenue DESC;

-- 2. Category Performance
SELECT 
    Category, 
    AVG(Sales) as Avg_Sale_Price, 
    SUM(Quantity) as Total_Quantity
FROM orders
GROUP BY Category;

-- 3. Overall Average
SELECT AVG(Sales) FROM orders;

# Part 3: Data Merging (Joins)
*Goal: Combining two tables based on a common column (The "VLOOKUP" logic).*

| Task | 🟢 Excel | 🔵 SQL | 🟡 Python (Pandas) |
| :--- | :--- | :--- | :--- |
| **Left Join** (Keep all Sales, bring Manager) | `VLOOKUP` or `XLOOKUP` | `LEFT JOIN` | `pd.merge(df1, df2, on='Key', how='left')` |
| **Inner Join** (Only matches) | Hard to do directly | `INNER JOIN` | `pd.merge(..., how='inner')` |
| **Full Join** (Keep everything) | Not possible directly | `FULL OUTER JOIN` | `pd.merge(..., how='outer')` |

In [ ]:
# Python Real Example: Merging Data
import pandas as pd
from IPython.display import display

# 1. Load Main Data (Sales)
df = pd.read_csv('ecommerce_data.csv')

# 2. Create a Second Dataset (Simulating a 'Managers' table)
# Notice: We have 'Frankfurt' here, but no 'Cologne'.
manager_data = {
    'City': ['Berlin', 'Munich', 'Hamburg', 'Frankfurt'], 
    'Region': ['North', 'South', 'North', 'West'],
    'Manager': ['Hans', 'Helga', 'Klaus', 'Petra']
}
df_managers = pd.DataFrame(manager_data)

print("--- Secondary Table (Managers) ---")
display(df_managers)

# 3. LEFT JOIN: Keep all Sales data, add Manager info where possible
# This is exactly like doing a VLOOKUP in Excel.
merged_df = pd.merge(df, df_managers, on='City', how='left')

print("\n--- Sales Data Combined with Manager Info (First 5 rows) ---")
display(merged_df[['Date', 'City', 'Sales', 'Manager', 'Region']].head())

# 4. Check for Missing Values (The "Real Case" Scenario)
# Since 'Cologne' is in Sales but NOT in Managers table, it should have NaN (Empty) values.
missing_manager = merged_df[merged_df['Manager'].isnull()]
print(f"\n--- Rows with Missing Manager (Cologne): {len(missing_manager)} rows ---")
display(missing_manager[['City', 'Sales', 'Manager']].head(3))

In [ ]:
-- SQL Equivalent

-- Joining Sales table (orders) with Managers table
-- We use LEFT JOIN to make sure we don't lose any sales records, even if a manager is missing.

SELECT 
    orders.Date,
    orders.City,
    orders.Sales,
    managers.Manager,
    managers.Region
FROM orders
LEFT JOIN managers ON orders.City = managers.City;

# Part 4: Data Cleaning & Text Manipulation
*Goal: Fixing messy data—handling missing values (NULLs) and standardizing text.*

| Task | 🟢 Excel | 🔵 SQL | 🟡 Python (Pandas) |
| :--- | :--- | :--- | :--- |
| **Check Missing** | Filter > Blanks | `WHERE Col IS NULL` | `df.isnull().sum()` |
| **Fill Missing** | Find & Select > Go To Special > Blanks | `COALESCE(Col, 'Value')` | `df['Col'].fillna('Value')` |
| **Drop Missing** | Delete Rows | `DELETE FROM table WHERE...` | `df.dropna()` |
| **Text Replace** | `SUBSTITUTE(Cell, "Old", "New")` | `REPLACE(Col, 'Old', 'New')` | `df['Col'].str.replace('Old', 'New')` |
| **Text Upper** | `UPPER(Cell)` | `UPPER(Col)` | `df['Col'].str.upper()` |

In [ ]:
# Python Real Example: Cleaning Data
import pandas as pd
from IPython.display import display

# 1. Load Data
df = pd.read_csv('ecommerce_data.csv')

# 2. Identify Missing Values (The "Dirty" Reality)
# We expect to see missing values in 'Category' because we created them that way.
print("--- 1. Count of Missing Values per Column ---")
print(df.isnull().sum())

# 3. Fill Missing Values (Imputation)
# Logic: If Category is empty, assume it is 'Other'
df['Category'] = df['Category'].fillna('Other')

print("\n--- 2. Verify Cleaning (First 5 rows - look at Category) ---")
# Notice row 0 to 4 might have been NaN, now they should be 'Other'
display(df.head())

# 4. Text Manipulation
# Logic: Let's standardize 'City' names to be all UPPERCASE to avoid duplicates like 'Berlin' vs 'berlin'
df['City'] = df['City'].str.upper()

print("\n--- 3. Standardized Text (Cities are now UPPERCASE) ---")
display(df[['Date', 'City', 'Sales']].head())

In [ ]:
--  SQL Equivalent

-- 1. Find rows with missing Category
SELECT * FROM orders 
WHERE Category IS NULL;

-- 2. Fill Missing Values (Display logic, not permanent change)
SELECT 
    Order_ID, 
    COALESCE(Category, 'Other') as Cleaned_Category 
FROM orders;

-- 3. Make City Uppercase
SELECT 
    Order_ID, 
    UPPER(City) as City_Upper 
FROM orders;

# Part 5: Date & Time Logic
*Goal: Time travel—extracting months, years, and calculating duration.*

| Task | 🟢 Excel | 🔵 SQL | 🟡 Python (Pandas) |
| :--- | :--- | :--- | :--- |
| **Convert to Date** | Change Cell Format | `CAST(col AS DATE)` | `pd.to_datetime(df['col'])` |
| **Extract Year** | `YEAR(Cell)` | `EXTRACT(YEAR FROM col)` | `df['col'].dt.year` |
| **Extract Month** | `MONTH(Cell)` | `EXTRACT(MONTH FROM col)` | `df['col'].dt.month` |
| **Day Difference** | `=Today - DateCell` | `CURRENT_DATE - col` | `(now - df['col']).dt.days` |
| **Filter by Date** | Filter > Date Filters | `WHERE col >= '2024-01-01'` | `df[df['col'] >= '2024-01-01']` |

In [ ]:
# Python Real Example: Working with Dates
import pandas as pd
from IPython.display import display

# 1. Load Data
df = pd.read_csv('ecommerce_data.csv')

# ⚠️ CRITICAL STEP: Convert 'Date' column from String (Text) to Datetime Object
# If you skip this, .dt accessor will not work!
df['Date'] = pd.to_datetime(df['Date'])

# 2. Extract Year and Month (For Reporting)
# We create new columns to analyze seasonality
df['Year'] = df['Date'].dt.year
df['Month_Name'] = df['Date'].dt.month_name()

print("--- 1. Data with new Year/Month Columns ---")
display(df[['Date', 'Year', 'Month_Name', 'Sales']].head())

# 3. Calculate "Days Since Purchase" (Recency)
# How many days have passed since the order? (Crucial for identifying churn)
today = pd.Timestamp.now()
df['Days_Since'] = (today - df['Date']).dt.days

print("\n--- 2. Days Since Transaction (Recency) ---")
display(df[['Date', 'Days_Since']].head())

# 4. Analysis: Total Sales by Month (Seasonality)
# Now we can answer: "Which month is our best month?"
monthly_sales = df.groupby('Month_Name')['Sales'].sum().sort_values(ascending=False)

print("\n--- 3. Best Performing Months ---")
display(monthly_sales.to_frame())

In [ ]:
-- SQL Equivalent

-- 1. Extract parts of the date
SELECT 
    Date,
    EXTRACT(YEAR FROM Date) as Order_Year,
    EXTRACT(MONTH FROM Date) as Order_Month
FROM orders;

-- 2. Calculate Age (Days Since)
-- Note: Syntax varies by SQL type (PostgreSQL uses AGE or simply subtraction)
SELECT 
    Date,
    CURRENT_DATE - Date as Days_Since
FROM orders;

-- 3. Monthly Sales Analysis
SELECT 
    EXTRACT(MONTH FROM Date) as Month, 
    SUM(Sales) as Total_Sales
FROM orders
GROUP BY EXTRACT(MONTH FROM Date)
ORDER BY Total_Sales DESC;

# Part 6: Conditional Logic (If/Else)
*Goal: Decision making—creating new categories based on rules.*

| Task | 🟢 Excel | 🔵 SQL | 🟡 Python (Pandas/Numpy) |
| :--- | :--- | :--- | :--- |
| **Simple IF** (2 Choices) | `IF(A2>100, "High", "Low")` | `CASE WHEN Sales > 100 THEN 'High' ELSE 'Low' END` | `np.where(df['Sales'] > 100, 'High', 'Low')` |
| **Multiple Conditions** (3+ Choices) | `IFS(A2>100,"High", A2>50,"Med", ...)` | `CASE WHEN... WHEN... ELSE... END` | `np.select([cond1, cond2], ['High', 'Med'], default='Low')` |
| **Complex Logic** (Custom Function) | VBA Macro | Stored Procedure | `df.apply(lambda x: my_func(x), axis=1)` |

In [ ]:
# Python Real Example: Customer Segmentation
import pandas as pd
import numpy as np  # <--- Essential for logic (np.where)
from IPython.display import display

# 1. Load Data
df = pd.read_csv('ecommerce_data.csv')

# 2. Simple Logic: "High Value" Flag
# If Sales > 300, it is a High Value Order, otherwise 'Standard'
df['Order_Type'] = np.where(df['Sales'] > 300, 'High Value', 'Standard')

print("--- 1. Simple Logic (High Value vs Standard) ---")
display(df[['Order_ID', 'Sales', 'Order_Type']].head())

# 3. Multiple Conditions: Customer Tiers (Gold, Silver, Bronze)
# Logic:
#   - Sales > 400  -> 'Gold'
#   - Sales > 200  -> 'Silver'
#   - Else         -> 'Bronze'

conditions = [
    (df['Sales'] > 400),
    (df['Sales'] > 200)
]
choices = ['Gold', 'Silver']

# np.select checks conditions in order. If none match, it uses 'default'
df['Tier'] = np.select(conditions, choices, default='Bronze')

print("\n--- 2. Advanced Logic (Gold/Silver/Bronze Tiers) ---")
display(df[['Order_ID', 'Sales', 'Tier']].head(10))

# 4. Analysis: How many orders do we have per Tier?
print("\n--- 3. Tier Distribution ---")
display(df['Tier'].value_counts().to_frame())

In [ ]:
-- SQL Equivalent

-- 1. Simple Case
SELECT 
    Order_ID, 
    Sales,
    CASE 
        WHEN Sales > 300 THEN 'High Value'
        ELSE 'Standard'
    END as Order_Type
FROM orders;

-- 2. Multiple Conditions (Tiers)
SELECT 
    Order_ID, 
    Sales,
    CASE 
        WHEN Sales > 400 THEN 'Gold'
        WHEN Sales > 200 THEN 'Silver'
        ELSE 'Bronze'
    END as Tier
FROM orders;

# 📘 Part 7: Statistics & Ranking (Window Functions)
*Goal: Advanced Analysis—Running totals, rankings, and comparing rows (e.g., Day vs Previous Day).*

| Task | 🟢 Excel | 🔵 SQL | 🟡 Python (Pandas) |
| :--- | :--- | :--- | :--- |
| **Running Total** (Cumulative) | Pivot: Show Values As > Running Total | `SUM(col) OVER (ORDER BY date)` | `df['col'].cumsum()` |
| **Ranking** (Leaderboard) | `RANK.EQ(cell, range)` | `RANK() OVER (ORDER BY col DESC)` | `df['col'].rank(ascending=False)` |
| **Lag / Shift** (Prev Row) | `=A3 - A2` | `LAG(col) OVER (ORDER BY date)` | `df['col'].shift(1)` |
| **Std Deviation** | `STDEV.S(range)` | `STDDEV(col)` | `df['col'].std()` |

In [ ]:
# Python Real Example: Advanced Stats & Window Functions
import pandas as pd
from IPython.display import display

# 1. Load Data & Ensure Date is sorted (Crucial for Running Totals)
df = pd.read_csv('ecommerce_data.csv')
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(by='Date') # Sort chronologically first!

# 2. Running Total (Cumulative Sum)
# Logic: "How much total revenue have we made up to this specific date?"
df['Cumulative_Sales'] = df['Sales'].cumsum()

print("--- 1. Running Total (Watch the Cumulative Column Grow) ---")
display(df[['Date', 'Sales', 'Cumulative_Sales']].head())

# 3. Ranking
# Logic: Rank orders from highest sales (1) to lowest.
df['Rank'] = df['Sales'].rank(ascending=False)

print("\n--- 2. Ranking (Top 3 Orders) ---")
display(df.sort_values('Rank').head(3)[['Date', 'Sales', 'Rank']])

# 4. Lag / Shift (Day-over-Day Growth)
# Logic: Compare current row's sales with the PREVIOUS row's sales.
df['Previous_Sales'] = df['Sales'].shift(1) # Moves everything down by 1
df['Growth'] = df['Sales'] - df['Previous_Sales']

print("\n--- 3. Previous Row Comparison (Lag) ---")
display(df[['Date', 'Sales', 'Previous_Sales', 'Growth']].head())

In [ ]:
-- SQL Equivalent (Window Functions)

-- 1. Running Total
SELECT 
    Date, 
    Sales, 
    SUM(Sales) OVER (ORDER BY Date) as Cumulative_Sales
FROM orders;

-- 2. Ranking
SELECT 
    Date, 
    Sales, 
    RANK() OVER (ORDER BY Sales DESC) as Sales_Rank
FROM orders;

-- 3. Lag (Previous Row)
SELECT 
    Date, 
    Sales,
    LAG(Sales) OVER (ORDER BY Date) as Previous_Sales
FROM orders;

---

# Conclusion & Key Takeaways

In this project, we simulated a complete Data Analysis lifecycle—from data generation to advanced statistical ranking—demonstrating how business logic translates across the three most essential tools in the industry.

### 🚀 Competencies Demonstrated:
1.  **Tool Agnosticism:** Ability to seamlessly switch between **Excel** (Logic), **SQL** (Querying), and **Python** (Automation) depending on the business need.
2.  **End-to-End Analysis:** Handled the full pipeline:
    * *Exploration:* Slicing and filtering raw data.
    * *Cleaning:* Handling missing values (`NULLs`) and standardizing text.
    * *Modeling:* Joining disparate datasets (Sales + Managers).
    * *Intelligence:* Creating customer tiers and analyzing time-series trends.
3.  **Technical Depth:** Utilized advanced techniques like **Window Functions** and **Vectorized Operations** (NumPy) for efficient processing.

### 💼 Business Impact:
This handbook serves not just as a code repository, but as a proof of concept that technical skills can be directly applied to solve business problems like **Churn Prediction**, **Sales Forecasting**, and **Customer Segmentation**.

---
*Thank you for reviewing my "Analyst's Rosetta Stone". Feel free to check out my other repositories or contact me for collaboration!*